# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP1 - Méthodes de recherche 
### Membres de l'équipe
    - William Harvey (1851388) 1
    - Claudia Onorato (1845448) 2
    - Mathieu Bélanger (1850591) 3




## Rush Hour

*RushHour* est un jeu dont le but est de libérer une voiture d'un parking. Les voitures bougent dans une grille 6x6 verticalement ou horizontalement. Chaque voiture occupe 2 ou 3 cases. Les voitures ne peuvent quitter le parking, à l'exception de la voiture rouge. Le but du jeu est donc de faire sortir la voiture rouge. L'image qui suit donne un exemple d'une configuration initiale.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

L'objectif de ce TP est d'écrire un programme qui trouve la solution de ce problème en un **minimum de mouvements**. Si vous voulez vous familiariser avec ce jeu, vous pouvez [jouer sur ce site](http://www.thinkfun.com/mathcounts/play-rush-hour). Il faut autoriser l'utilisation de Flash pour accéder au jeu.

Pour le TP, il faudra compléter les deux classes fournies en Python. Vous pouvez réimplémenter ces classes-là et faire le TP dans un autre langage si vous le désirez.

## 1. Représentation du problème (15pts)

Le problème est représenter selon les règles suivantes. Les six colonnes sont numérotées de gauche à droite de 0 à 5 et les 6 lignes sont numérotées de haut en bas, de 0 à 5 aussi.  **Par convention la voiture rouge a pour index 0 et est de longueur 2. La sortie du parking est comme sur la figure et se trouve donc vis-à-vis la case 5-2**. La voiture rouge est considérée à l'extérieur lorsqu'une partie de la voiture est hors de la grille.

L'état du parking au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos`: un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente lapremière case occupée par la voiture selon l'orientation de la voiture);
- `c`: l'index de la voiture deplacée pour atteindre l'état courant à partir de l'état précédent;
-`d`: la direction du déplacement fait à partir de l'état précédent pour atteindre l'état courant (+1: vers la droite ou vers le bas, -1: vers la gauche ou vers le haut);
-`prev`: l'état précédent


In [0]:
import numpy as np
import math
import copy

# Nb columns and nb lines are the same
NB_SQUARES = 6

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i (première case occupée par la voiture);

        Où la première case correspond à la case ayant le plus petit index
        (et non au devant/derrière de l'auto)
        """
        self.pos = np.array(pos)
        
        """
        c, d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.h = 0

    """
    Constructeur d'un état à partir mouvement (c,d)
    Où c: index de la voiture à déplacer
       d: direction du mouvement (+1 pour la droite/bas, -1 pour la gauche/haut)
    """
    def move(self, c, d):
        # TODO
        next_position = np.array(self.pos)
        next_position[c] += d
  
        nextState = State(next_position)
        nextState.prev = self
        nextState.c = c
        nextState.d = d
        nextState.nb_moves = self.nb_moves + 1
        return nextState


    """ est il final? """
    def success(self):
        # TODO
        return self.pos[0] == NB_SQUARES - 2
    

    """
    Estimation du nombre de coup restants 
    """
    def estimee1(self):
        # TODO
        final_state_red_car_pos = NB_SQUARES - 2
        return final_state_red_car_pos - self.pos[0]

    def estimee2(self, rh):
        # TODO
        rh.init_positions(self)

        # check how many squares are not free from the car position and the exit knowing 
        # it moves horizontally on y=rh.move_on[0] between x=self.pos[0]+rh.length[0] to NB_SQUARES-1

        line = rh.move_on[0]
        nb_occupied_squares = 0
        for column in range(self.pos[0]+rh.length[0], NB_SQUARES):
          nb_occupied_squares += int(not rh.free_pos[line, column])

        return self.estimee1() + nb_occupied_squares
      
    def estimee3(self, rh):
      rh.init_positions(self)

      # check how many squares are not free from the car position and the exit knowing 
      # If a square is occupied by a truck and the truck is centered on the exit path,
      # it has to at least move two times. 

      line = rh.move_on[0] # always on the 3rd line
      nb_moves = 0
      for column in range(self.pos[0]+rh.length[0], NB_SQUARES):
        if not rh.free_pos[line, column]:
          # If there's a car in the way, add one move to do
          nb_moves += 1
          truck_indexes = [i for i, pos in enumerate(self.pos) if pos == line - 1]
          for truck_index in truck_indexes:
            # Add another move to do if it's a truck that has to move 2 times to get out of the way
            nb_moves += 1 if rh.length[truck_index] == 3 and rh.move_on[truck_index] == column else 0

      return self.estimee1() + nb_moves
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
        
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.nb_moves + self.h) < (other.nb_moves + other.h)

#### Implémentation 
   - Complétez la méthode ***success*** de la classe **State** qui indique si un état est final. Cette méthode vérifie si la voiture rouge peut sortir le prochain coup.

   - Complétez la fonction ***move*** en déplaçant la voiture c de 1 case dans la bonne direction (d=+1 ou -1). Nous supposons que le déplacement est possible lorsque la fonction est appelée. **Attention** : Cette fonction doit créer un nouvel objet State et définir les variables prev, c et d.
   
Vous pouvez tester votre code avec la fonction ***test1()*** dans la cellule suivante. Assurez-vous d'avoir déjà roulé la cellule avec la définition de la classe **State** avant. Vous devriez obtenir les résultats suivants:
```
True
True
0 1
True
True
```

In [0]:
def test1():
    positioning = [1, 0, 1, 4, 2, 4, 0, 1]
    s0 = State(positioning)
    b = not s0.success()
    print(b)
    s = s0.move(1,1)
    print(s.prev == s0)
    b = b and s.prev == s0
    print(s0.pos[1], " ", s.pos[1])
    s = s.move(6,1)
    s = s.move(1,-1)
    s = s.move(6,-1)
    print(s == s0)
    b = b and s == s0
    s = s.move(1,1)
    s = s.move(2,-1)
    s = s.move(3,-1)
    s = s.move(4,1)
    s = s.move(4,-1)
    s = s.move(5,-1)
    s = s.move(5,1)
    s = s.move(5,-1)
    s = s.move(6,1)
    s = s.move(6,1)
    s = s.move(6,1)
    s = s.move(7,1)
    s = s.move(7,1)
    s = s.move(0,1)
    s = s.move(0,1)
    s = s.move(0,1)
    print(s.success())
    b = b and s.success()    
    print("\n", "résultat correct" if b else "mauvais résultat")
    
test1()

True
True
0   1
True
True

 résultat correct


## 2. Mouvements possibles (15pts)
Pour représenter le problème, nous allons utiliser la classe **RushHour** qui contient les champs suivants:

- `nbcars`: le nombre de voitures dans la grille;
- `horiz`: un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on`: un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color`: un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos`: une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouve dans le même index dans chacun des vecteurs.

In [0]:
from collections import deque
import heapq

class Rushhour:
    
  def __init__(self, horiz, length, move_on, color=None):
    self.nbcars = len(horiz)
    self.horiz = horiz
    self.length = length
    self.move_on = move_on
    self.color = color
    
    self.free_pos = None
  
  def init_positions(self, state):
    # TODO
    self.free_pos = np.ones(shape=(NB_SQUARES,NB_SQUARES),dtype=bool)

    for car_index, car_first_pos in enumerate(state.pos):
      # for each square the car occupies, set the square to false
      car_is_horiz = self.horiz[car_index]
      car_move_on = self.move_on[car_index]

      for i in range(0, self.length[car_index]):
        if car_is_horiz:
          self.free_pos[car_move_on][car_first_pos + i] = False
        else:
          self.free_pos[car_first_pos + i][car_move_on] = False


  def possible_moves(self, state):
    self.init_positions(state)
    new_states = []
    # TODO
    for car_index, car_first_pos in enumerate(state.pos):
      car_move_on = self.move_on[car_index]
      car_length = self.length[car_index]
      is_horiz = self.horiz[car_index]
    
      # Check if we can back up one square
      if car_first_pos != 0 and self._is_square_free(car_move_on, car_first_pos - 1, is_horiz):
        new_states.append(state.move(car_index, -1))
      
      # Check if we can go forward one square
      if car_first_pos + car_length < NB_SQUARES and self._is_square_free(car_move_on, car_first_pos + car_length, is_horiz):
        new_states.append(state.move(car_index, 1)) 

    return new_states
  
  def solve(self, state):
    """
    Breadht first search with initial state
    """

    visited = set()
    fifo = deque([state])
    visited.add(state)
    # TODO
    nb_visited_states = 0

    while len(fifo) != 0:
      state = fifo.popleft()

      if state.success():
        print("Solution found. %i states have been visited." % (nb_visited_states))
        return state
      else:
        next_states = set(self.possible_moves(state)) - visited
        fifo.extend(next_states)
        visited.update(next_states)
      
      nb_visited_states += 1

    print("No solution found")
    return None
  
                  
  def solve_Astar(self, state):
    visited = set()
    visited.add(state)
    
    priority_queue = []
    state.h = state.estimee3(self) # state.estimee1()
    heapq.heappush(priority_queue, state)
    # TODO
    nb_visited_states = 0

    while len(priority_queue) != 0:
      state = heapq.heappop(priority_queue)

      if state.success():
        print("Solution found. %i states have been visited." % (nb_visited_states))
        return state
      else:
        next_states = set(self.possible_moves(state)) - visited
        for next_state in next_states:
          next_state.h = next_state.estimee3(self) # next_state.estimee1() 
          heapq.heappush(priority_queue, next_state)
        visited.update(next_states)

      nb_visited_states += 1

    print("No solution found")

    return None
  
                  
  def print_solution(self, state):
    # TODO
    direction_labels = [["la droite", "la gauche"], ["le bas", "le haut"]]

    if state.prev is not None:
      self.print_solution(state.prev)

      direction = 0 if state.d == 1 else 1 # Converts state.d from -1 to 1 (go up or go left) and from 1 to 0 (go down or go right)
      direction_label = direction_labels[0][direction] if self.horiz[state.c] else direction_labels[1][direction]

      print("%i. Voiture %s vers %s" % (int(state.nb_moves), self.color[state.c], direction_label))

    return 0

  def _is_square_free(self, x, y, is_horizontal):
    """
    Checks if the x line and y column indexed if horizontal or
    with y line and x column indexed square is a free space. 
    """
    x_index = x if is_horizontal else y
    y_index = y if is_horizontal else x

    return self.free_pos[x_index, y_index]

#### Implémentation 
   - Nous voulons utiliser une matrice 6x6 `free_pos` indiquant les cases du parking qui sont libres. Complétez la méthode **init_positions(State)** qui initialise la matrice `free_pos` en fonction de l'état donné avec les conventions que *True* signifie que la cases est libre et que `free_pos[i][j]` représente la case (ligne,colonne) du parking.

Rouler la cellule suivante pour tester votre code. Elle testera la fonction ***init_positions(s)***. Vous devriez obtenir les résultats suivants:
```
[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
```

In [0]:
def test2():
    rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3])
    s = State([1, 0, 1, 4, 2, 4, 0, 1])
    rh.init_positions(s)
    
    print(rh.free_pos)
    ans = [[False,False,True,True,True,False],[False,True,True,False,True,False],[False,False,False,False,True,False],
           [False,True,True,False,True,True],[False,True,True,True,False,False],[False,True,False,False,False,True]]
    
    # b = True # version originale du prof. Retourne toujours vrai
    # b = b and (rh.free_pos[i,j] == ans[i,j] for i in range(6) for j in range(6))
    b = all([rh.free_pos[i][j] == ans[i][j] for i in range(6) for j in range(6)])
    print("\n", "résultat correct" if b else "mauvais résultat")

test2()

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]

 résultat correct


#### Implémentation 
   - Complétez la méthode **possible_moves(state)**  qui renvoie l'ensemble d'états qui peuvent être atteints à partir de l'état *state*. Cet ensemble est représenté par une list de states; l'ordre dans la liste n'est pas important.

Rouler la cellule suivante pour tester votre code. Elle testera la fonction ***possible_moves(state)***. Vous devriez obtenir les résultats suivants:
```
5
6
```
Il y a 5 mouvements possibles à partir de l'état représenté par l'image au début du TP. Il y a 6 mouvements possibles à partir de l'état suivant où la voiture bleu s'est déplacée d'une case vers la droite.

In [0]:
def test3():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    s2 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 2])
    print(len(rh.possible_moves(s)))
    print(len(rh.possible_moves(s2)))

print("Result should be: 5, 6")
test3()

Result should be: 5, 6
5
6


## 3. À la recherche d'une solution (25pts)

Il est maintenant possible de chercher une solution avec les implémentations faites. Il y a deux idées principales: 

1. Nous cherchons la solution la plus rapide, il nous faut énumérer tous les états pour les ordonner en ordre croissant en terme de nombre de mouvements. On va donc utiliser une file de type FIFO. 
2. Il est nécessaire de mémoriser les états que l'on a déjà trouvé. Pour cela on va utiliser un ensemble hash. Il s'agit d'utiliser une variable locale **visited** de type *set*.

On peut représenter un ensemble d'état par un arbre dont la racine est l'état initial et les nœuds correspondent aux états `s`. La relation entre chaque état est de type `s` *est fils de* `p`, où `s` et `p` sont deux états. Autrement dit, `s` peut être obtenu à partir de `p` en un mouvement.

On cherche à faire une recherche en largeur sur cet arbre. On peut donc représenter la file par une liste (FIFO). La file commence avec l'état initial. Tant que la liste n'est pas vide, on extrait le premier état. S'il est final, on termine l'algorithme. Dans le cas contraire, on ajoute ses fils non visités à la fin de la liste. On ajoute ensuite cet état dans la liste `visited`.

#### Implementation 
   - Complétez la méthode **solve** en implémentant cet algorithme. La méthode doit renvoyer un état final (la voiture rouge doit être juste devant la sortie). 
   - Complétez la méthode **printSolution(state)** qui affiche une solution à partir d'un état final `state`. Il faut afficher les mouvements des voitures dans le bon ordre.

Utilisez la cellule suivante pour tester votre code. Vous devriez obtenir le résultat suivant:  
Il y a 46 mouvement à faire pour résoudre le problème de la figure de départ. **Note** Votre solution peut être différente de celle présentée ici, cela dépend de la méthode de construction de la liste dans la méthode ``possible_moves``. Cependant votre solution doit avoir le même nombre de mouvements (46).

<p style="font-size: 10pt">

```
1. Voiture orange vers le haut
2. Voiture rouge vers la gauche
3. Voiture vert clair vers le bas
4. Voiture vert clair vers le bas
5. Voiture jaune vers la gauche
6. Voiture jaune vers la gauche
7. Voiture vert vers le haut
8. Voiture noir vers la droite
9. Voiture vert clair vers le bas
10. Voiture rouge vers la droite
11. Voiture orange vers le bas
12. Voiture jaune vers la gauche
13. Voiture violet clair vers le haut
14. Voiture violet vers la gauche
15. Voiture beige vers le haut
16. Voiture beige vers le haut
17. Voiture noir vers la droite
18. Voiture bleu vers la droite
19. Voiture rose vers le bas
20. Voiture bleu vers la droite
21. Voiture vert clair vers le bas
22. Voiture violet vers la gauche
23. Voiture violet vers la gauche
24. Voiture violet clair vers le bas
25. Voiture violet clair vers le bas
26. Voiture bleu ciel vers la gauche
27. Voiture bleu ciel vers la gauche
28. Voiture bleu ciel vers la gauche
29. Voiture violet clair vers le haut
30. Voiture violet clair vers le haut
31. Voiture violet vers la droite
32. Voiture rose vers le haut
33. Voiture violet vers la droite
34. Voiture vert clair vers le haut
35. Voiture beige vers le haut
36. Voiture violet vers la droite
37. Voiture violet clair vers le bas
38. Voiture violet clair vers le bas
39. Voiture beige vers le haut
40. Voiture bleu vers la gauche
41. Voiture bleu vers la gauche
42. Voiture bleu vers la gauche
43. Voiture violet clair vers le bas
44. Voiture rouge vers la droite
45. Voiture rouge vers la droite
46. Voiture rouge vers la droite
```
    
</p>

In [0]:
def solve46(solve_with_a_star=False):
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    if solve_with_a_star:
      s = rh.solve_Astar(s)
    else:
      s = rh.solve(s)
    rh.print_solution(s)

def solve16(solve_with_a_star=False):
    rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert clair", "violet", "orange", "vert", "bleu ciel", "jaune", "bleu"])
    s = State([1, 0, 1, 4, 2, 4, 0, 1])
    if solve_with_a_star:
      s = rh.solve_Astar(s)
    else:
      s = rh.solve(s)
    n = rh.print_solution(s)
    
def solve81(solve_with_a_star=False):
    rh = Rushhour([True, False, True, False, False, False, False, True, False, False, True, True, True],
                 [2, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2],
                 [2, 0, 0, 4, 1, 2, 5, 3, 3, 2, 4, 5, 5],
                 ["rouge", "jaune", "vert clair", "orange", "bleu clair", "rose", "violet clair","bleu", "violet", "vert", "noir", "beige", "jaune clair"])
    s = State([3, 0, 1, 0, 1, 1, 1, 0, 3, 4, 4, 0, 3])
    if solve_with_a_star:
      s = rh.solve_Astar(s)
    else:
      s = rh.solve(s)
    n = rh.print_solution(s)

In [0]:
solve46()
print("\n--------------------------------------------\n")
%time

Solution found. 4750 states have been visited.
1. Voiture vert vers le haut
2. Voiture bleu vers la droite
3. Voiture noir vers la droite
4. Voiture orange vers le haut
5. Voiture rouge vers la gauche
6. Voiture rose vers le bas
7. Voiture vert clair vers le bas
8. Voiture vert clair vers le bas
9. Voiture vert clair vers le bas
10. Voiture rouge vers la droite
11. Voiture orange vers le bas
12. Voiture jaune vers la gauche
13. Voiture jaune vers la gauche
14. Voiture jaune vers la gauche
15. Voiture violet clair vers le haut
16. Voiture violet vers la gauche
17. Voiture beige vers le haut
18. Voiture bleu vers la droite
19. Voiture vert clair vers le bas
20. Voiture violet vers la gauche
21. Voiture beige vers le haut
22. Voiture violet vers la gauche
23. Voiture noir vers la droite
24. Voiture violet clair vers le bas
25. Voiture violet clair vers le bas
26. Voiture bleu ciel vers la gauche
27. Voiture bleu ciel vers la gauche
28. Voiture bleu ciel vers la gauche
29. Voiture beige ve

#### Tests
Voici deux autres problèmes à résoudre : le premier doit être résolu en 16 mouvement; le second en 81.

In [0]:
solve16()
print("\n--------------------------------------------\n")
solve81()
print("\n--------------------------------------------\n")
%time

Solution found. 1071 states have been visited.
1. Voiture vert vers la gauche
2. Voiture vert clair vers la droite
3. Voiture bleu ciel vers la gauche
4. Voiture violet vers le haut
5. Voiture jaune vers le bas
6. Voiture orange vers le haut
7. Voiture vert vers la gauche
8. Voiture bleu ciel vers la gauche
9. Voiture bleu ciel vers la gauche
10. Voiture bleu vers le bas
11. Voiture bleu vers le bas
12. Voiture rouge vers la droite
13. Voiture rouge vers la droite
14. Voiture jaune vers le bas
15. Voiture jaune vers le bas
16. Voiture rouge vers la droite

--------------------------------------------



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: RuntimeWarning: overflow encountered in long_scalars


Solution found. 3014 states have been visited.
1. Voiture violet clair vers le haut
2. Voiture jaune clair vers la droite
3. Voiture violet vers le bas
4. Voiture bleu vers la droite
5. Voiture bleu vers la droite
6. Voiture jaune vers le bas
7. Voiture vert clair vers la gauche
8. Voiture bleu clair vers le bas
9. Voiture bleu clair vers le bas
10. Voiture bleu vers la droite
11. Voiture vert vers le haut
12. Voiture beige vers la droite
13. Voiture rose vers le haut
14. Voiture rouge vers la gauche
15. Voiture rouge vers la gauche
16. Voiture jaune vers le bas
17. Voiture orange vers le bas
18. Voiture jaune vers le bas
19. Voiture rouge vers la gauche
20. Voiture rose vers le bas
21. Voiture vert clair vers la droite
22. Voiture vert clair vers la droite
23. Voiture vert clair vers la droite
24. Voiture rose vers le haut
25. Voiture rouge vers la droite
26. Voiture rouge vers la droite
27. Voiture bleu clair vers le haut
28. Voiture bleu clair vers le haut
29. Voiture bleu clair ver

## 4. A* algorithme (35pts)

La méthode proposée précédemment utilise une recherche en largeur pour trouver la solution. Une solution plus rapide peut être mise en place avec un algorithme `A*`. L'idée principale de l'algorithme A* est d'insérer de *l'intelligence artificielle* à la méthode de résolution. Ceci est fait en utilisant une fonction d'estimation qui permet d'évaluer la distance entre un état et la solution. 

Au lieu d'utiliser une approche *first-in first-out (premier entré, premier sorti)* lors de la visite des états, l'algorithme A* défini une valeur de priorité pour chaque état, à partir de la fonction d'estimation, et commence par visiter les états les plus prioritaires. Pour cela, on utilise une structure de données bien plus appropriée : la file de priorité.

En plus de la fonction d'estimation, A* doit aussi tenir compte d'une autre information importante : le coût actuel de l'état. Ce coût est tout simplement, dans notre cas, la distance entre l'état actuel et l'état initial. Ces deux éléments mis ensembles permettent de calculer la fonction d'estimation heuristique ***f*** d'un état `s`, comme la somme du coût actuel de l'état ***c*** et de l'estimation du coût restant ***h***. 

<center> f(s) = coût(s) + h(s) </center>

**Mise en place de l'algorithme**

- Il nous faut donc calculer le nombre de mouvements nécessaires pour atteindre l'état actuel, `coût(s)`. Pour le calculer, créez une variable `nb_moves` dans la classe `State` qui enregistre le nombre de mouvements effectués pour arriver à cet état. Cette variable commence à 0 et doit être incrémentée à chaque fois que l'on crée un nouvel état en utilisant la fonctione `move(c, d)`.

Le coût de la fonction d'estimation est représenté par la variable `h`. Cette fonction estime le nombre de coups restants. Cette composante joue un rôle important dans l'efficacité de l'algorithme A*. Cette variable est utilisée dans la fonction `__lt__` de la classe State, qui compare la fonction estimative` f` de deux états. 

Il est donc important de créer une bonne fonction pour résoudre le problème. Plus cette fonction est efficace, moins d'états inutiles seront visités. Cette fonction doit par contre toujours renvoyer un minorant du nombre de coups restant. On va analyser deux fonctions différentes dans ce TP.

#### Fonction 1: à partir de la distance entre la voiture Rouge et la sortie.
- Complétez dans la classe State, la fonction ``estimee1()`` qui renvoie la distance entre la voiture rouge et la sortie.
- Maintenant complétez la méthode `solve_Astar(state)` pour résoudre le problème de la même façon qu'au début du TP, mais en utilisant la file de priorité à la place d'une file ordinaire.
- Modifiez les fonctions `solve_Astar(state)` et `solve(state)` de façon à compter et afficher le nombre d'états visités lors de la résolution.
- Modifiez le code des fonctions `solve46(),solve16(),solve81()` pour résoudre les problèmes à l'aide de l'algorithme A*. Vérifiez que les solutions sont obtenues en autant de mouvements qu'avec le premier algorithme, mais avec un nombre inférieur d'Etats visités. 

N.B: Vous pouvez ajouter des cellules pour faire le test ou afficher les resultats des algorithmes dans une même cellule. Veuillez bien identifier les résultats affichés.

#### Fonction 2:  à partir de la distance de la voiture rouge à la sortie et du nombre de voitures qui lui barrent le passage.
- Complétez dans la classe State, la fonction `estimee2` qui renvoie la distance entre la voiture rouge et la sortie plus le nombre de voitures entre la voiture rouge et la sortie.  Utilisez le paramètre `rh` fourni de la classe RushHour pour accéder à la position des voitures dans la classe State. **Attention:** Il n'est pas possible d'utiliser directement la matrice `free_pos`, car celle-ci peut ne pas être mise à jour avec l'état en cours d'estimation.
- Modifiez la fonction `solve_Astar` pour utiliser `estimee2()`.
- Utiliser les exemples fournis (`solve46(), solve16()` et `solve81())` pour tester cet algorithme et le comparer aux précédents. Comparez le nombre d'état visités par chacun des algorithmes pour le problèmes fournis.

N.B: Vous pouvez ajouter des cellules pour faire le test ou afficher les resultats des algorithmes dans une même cellule. Veuillez bien identifier les résultats affichés.



In [0]:
solve46(solve_with_a_star=True)
print("\n--------------------------------------------\n")
%time

Solution found. 3330 states have been visited.
1. Voiture orange vers le haut
2. Voiture noir vers la droite
3. Voiture rouge vers la gauche
4. Voiture vert clair vers le bas
5. Voiture vert clair vers le bas
6. Voiture vert clair vers le bas
7. Voiture rouge vers la droite
8. Voiture jaune vers la gauche
9. Voiture jaune vers la gauche
10. Voiture orange vers le bas
11. Voiture jaune vers la gauche
12. Voiture violet clair vers le haut
13. Voiture violet vers la gauche
14. Voiture beige vers le haut
15. Voiture bleu vers la droite
16. Voiture bleu vers la droite
17. Voiture rose vers le bas
18. Voiture vert clair vers le bas
19. Voiture violet vers la gauche
20. Voiture violet vers la gauche
21. Voiture beige vers le haut
22. Voiture noir vers la droite
23. Voiture violet clair vers le bas
24. Voiture violet clair vers le bas
25. Voiture bleu ciel vers la gauche
26. Voiture bleu ciel vers la gauche
27. Voiture bleu ciel vers la gauche
28. Voiture violet clair vers le haut
29. Voiture 

In [0]:
solve16(solve_with_a_star=True)
print("\n--------------------------------------------\n")
%time

Solution found. 851 states have been visited.
1. Voiture bleu ciel vers la gauche
2. Voiture bleu ciel vers la gauche
3. Voiture bleu ciel vers la gauche
4. Voiture bleu vers le bas
5. Voiture vert clair vers la droite
6. Voiture violet vers le haut
7. Voiture orange vers le haut
8. Voiture vert vers la gauche
9. Voiture vert vers la gauche
10. Voiture bleu vers le bas
11. Voiture rouge vers la droite
12. Voiture rouge vers la droite
13. Voiture jaune vers le bas
14. Voiture jaune vers le bas
15. Voiture jaune vers le bas
16. Voiture rouge vers la droite

--------------------------------------------

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [0]:
solve81(solve_with_a_star=True)
print("\n--------------------------------------------\n")
%time

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: RuntimeWarning: overflow encountered in long_scalars


Solution found. 2652 states have been visited.
1. Voiture violet clair vers le haut
2. Voiture jaune clair vers la droite
3. Voiture violet vers le bas
4. Voiture bleu vers la droite
5. Voiture bleu vers la droite
6. Voiture jaune vers le bas
7. Voiture jaune vers le bas
8. Voiture bleu clair vers le bas
9. Voiture vert clair vers la gauche
10. Voiture bleu clair vers le bas
11. Voiture bleu vers la droite
12. Voiture vert vers le haut
13. Voiture beige vers la droite
14. Voiture rose vers le haut
15. Voiture jaune vers le bas
16. Voiture rouge vers la gauche
17. Voiture rouge vers la gauche
18. Voiture rouge vers la gauche
19. Voiture rose vers le bas
20. Voiture vert clair vers la droite
21. Voiture vert clair vers la droite
22. Voiture orange vers le bas
23. Voiture vert clair vers la droite
24. Voiture rose vers le haut
25. Voiture rouge vers la droite
26. Voiture rouge vers la droite
27. Voiture bleu clair vers le haut
28. Voiture jaune vers le haut
29. Voiture bleu clair vers le 

## 5. Amélioration de l'heuristique (10pts)
Proposez une autre fonction heuristique pour le problème et tester là avec les exemples fournis (`solve46(), solve16()` et `solve81())`. Ajouter une fonction nommée `estimee3` et implémenter votre heuristique dedans. **Justifier votre choix d'heuristique.**

Voici les résultats comparant le nombre de noeuds visités selon la méthode utilisée.

|Nombre de noeuds visités     | BFS | A* estimée1 | A* estimée2 | A* estimée3 |
|------------|----|-----|---|---|
| solve46 | 4750  | 4041   | 3639 |3330 |
| solve16 | 1071  | 1048   | 969 |851 |
| solve81 | 3014  | 2962   | 2681 |2652 |


**Fonction 3:**

Notre choix d'heuristique pour la troisième estimée se base sur la deuxième méthode d'estimation. En effet, nous regardons encore le nombre de cases occupées entre la voiture rouge et la sortie, et nous y ajoutons le nombre de cases qui séparent la voiture rouge de la sortie. De plus, nous considérons qu'un camion (de taille 3) situé verticalement entre la voiture rouge et la sortie et "centré" sur la ligne de la voiture rouge devra effectué **au moins deux mouvements**. Le cas considéré est illustré sur l'image ci-haut avec le **camion violet**.

Nous voyons, selon le tableau ci-haut, que nous devons visiter moins d'états avant de trouver la solution au problème dans les trois cas avec notre nouvelle heuristique. 

## 6.  Directives de remise
Le travail sera réalisé en équipe de trois. Vous remettrez ce fichier nommés TP1\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boite de remise sur moodle. 

Tout devra être remis avant le **6 octobre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barême
Partie 1: 15 points

Partie 2: 15 points

Partie 3: 25 points

Partie 4: 35 points

Partie 5: 10 points

Pour un total de 100 points sur 100 points.


*Ce TP a été imaginé par Steve Oudot et Jean-Christophe Filliâtre (Ecole polytechnique, France) et complété
par Pierre Hulot et Rodrigo Randel.